<a href="https://colab.research.google.com/github/rauldoe/cpsc585proj3/blob/master/project3_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

%tensorflow_version 2.x
# Comment it out if not connecting to Google Drive
from google.colab import drive
import csv
import timeit

from __future__ import print_function

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

# Importing the EMNIST letters
from scipy import io as sio

def checkDevice():
  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

def testSpeedUp():
  # We run each op once to warm up; see: https://stackoverflow.com/a/45067900
  cpu()
  gpu()

  # Run the op several times.
  print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
        '(batch x height x width x channel). Sum of ten runs.')
  print('CPU (s):')
  cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
  print(cpu_time)
  print('GPU (s):')
  gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
  print(gpu_time)
  print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

def connectToCloudDrive():
  drive.mount('/content/drive', force_remount=True)

def disconnectFromCloudDrive():
  drive.flush_and_unmount()

def getCloudDriveBase():
  return '/content/drive/My Drive/Colab Notebooks/'

def testCsv(csvPath):
  with open(csvPath, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in reader:
      print(', '.join(row))

def testCsvOnCloudDrive(filename):
  connectToCloudDrive()
  filePath = getCloudDriveBase() + filename;
  testCsv(filePath)
  disconnectFromCloudDrive()

In [0]:

# Only run this cell to test the GPU vs CPU
checkDevice()
testSpeedUp()


In [0]:

# Only run this cell to test the Cloud Drive
testCsvOnCloudDrive('california_housing_test.csv')


In [0]:

filename = 'emnist-letters.mat'

# Use this for local
# filePath = './' + filename

# Use this for Google Colab
# filePath = '/content/' + filename

# Use this for cloud drive
connectToCloudDrive()
filePath = getCloudDriveBase() + filename;


Mounted at /content/drive


In [0]:
# Batch size of 128 had about 90.4% accuracy.
# Thus, a batch size of 1000 was used where accuracy was about 91.5%. 
# Signifigantly higher batch sizes also decreased test accuracy.
# A batch size of 104,000 led to an accuracy of about 
batch_size = 2000
# num_classes = 10
num_classes = 26
epochs = 1000 #There is early stopping, so it won't reach 1000 epochs. This needs to be high.

img_rows, img_cols = 28, 28

# https://stackoverflow.com/questions/51125969/loading-emnist-letters-dataset/53547262#53547262
mat = sio.loadmat(filePath)
data = mat['dataset']

x_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
x_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

val_start = x_train.shape[0] - x_test.shape[0]
x_val = x_train[val_start:x_train.shape[0],:]
y_val = y_train[val_start:x_train.shape[0]]
x_train = x_train[0:val_start,:]
y_train = y_train[0:val_start]

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train - 1, num_classes, dtype='float32')
y_test = tf.keras.utils.to_categorical(y_test - 1, num_classes, dtype='float32')

y_val = tf.keras.utils.to_categorical(y_val - 1, num_classes, dtype='float32')

# model = Sequential()
# # Sigmoid seemed to work better for test accuracy compared to relu. (sigmoid was getting 91% test accuracy compared to 89% for relu.)
# # Sigmoid was slighly better than tanh, but both were about the same test accuracy (within a few tenths of a percent)
# model.add(Dense(512, activation='sigmoid', input_shape=(784,)))
# # Tried different dropout rates, but 0.2 seemed to work well and provided a modest improvement.
# # (~0.5% test accuracy improvement compared to not using dropout at all)
# model.add(Dropout(0.2))
# # Compared to other numbers of neurons, this number seemed to work well (2000 hidden neurons)
# model.add(Dense(2000, activation='sigmoid'))
# model.add(Dropout(0.2))
# model.add(Dense(num_classes, activation='softmax'))

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.0001, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs, callbacks=[earlyStop],
                    validation_data=(x_val, y_val)
                    )
score = model.evaluate(x_test, y_test, verbose=0)



print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (104000, 28, 28, 1)
104000 train samples
20800 test samples
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               1179776   
_____________________________________________

In [0]:
# Finalization steps

disconnectFromCloudDrive()